In [2]:
## test Bsyesian optimization
from sklearn import metrics
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import catboost as cb
from bayes_opt import BayesianOptimization


In [ ]:
class optimizationClass():
    # 初始化的时候把数据集传进来
    def __init__(self,df):
        self.df = df 
        
     # 定义一个评估指标的函数评估模型好坏   
    def reg_calculate(self,true, prediction): 
        mse = metrics.mean_squared_error(true, prediction)
        rmse = np.sqrt(mse)
        mae = metrics.mean_absolute_error(true, prediction)
        mape = np.mean(np.abs((true - prediction) / true)) * 100
        r2 = metrics.r2_score(true, prediction)
        rmsle = np.sqrt(metrics.mean_squared_log_error(true, prediction))
        # print("mse: {}, rmse: {}, mae: {}, mape: {}, r2: {}, rmsle: {}".format(mse, rmse, mae, mape, r2, rmsle))
        # return mse, rmse, mae, mape, r2, rmsle
        return {"mse": mse, "rmse": rmse, "mae": mae, "mape": mape, "r2": r2, "rmsle": rmsle}    
    
    # 优化函数 参数为你需要调参的 hyper Parameter 这里用catBoost作为样例
    def optimization_function(self,iterations,learning_rate,depth,l2_leaf_reg):
        parameterDict = {"iterations":int(iterations),"learning_rate": float(learning_rate), "depth": int(depth),
                         "l2_leaf_reg":float(l2_leaf_reg),
                         "task_type":"CPU","logging_level":"Silent"}
        CB_Regressor=cb.CatBoostRegressor(**parameterDict)
        CB_Regressor.fit(self.df["X_train"], self.df["Y_train"])
        Y_pre=CB_Regressor.predict(self.df["X_test"])
        resDict =self.reg_calculate(self.df["Y_test"],Y_pre)
        return resDict["r2"]
    
    # 定义一下模型
    def run(self, init_points=2,n_iter=3):
        cb_bo = BayesianOptimization(
        self.optimization_function,
        {'iterations': (200, 5000),
        'learning_rate': (1e-6, 1e-2),
        'depth': (2, 15),
        'l2_leaf_reg': (0, 5)}
        )
        cb_bo.maximize(
                        init_points=init_points,
                        n_iter=n_iter)
        print("Final result:", cb_bo.max)
#     


In [ ]:
Opt = optimizationClass(df = {"X_train":X_Train, "X_test":X_Test, "Y_train":Y_Train, "Y_test":Y_Test})
Opt.run()

In [ ]:
## grid search

In [ ]:
## XGBoost
## test random forest